In [1]:
from z3 import *

In [2]:
x0, x1, y0, y1 = z3.Ints("x0 x1 y0 y1")
N = 1351235
M = N * 2

In [3]:
init = z3.And(x0 == 0, y0 == 0)
inductive_step = z3.And(x1 == x0 + 1, y1 == If(x0 % 2 == 0, y0 + 1, y0))
bad = z3.And(x0 == M, z3.Not(y0 == N))

(1 ≤ x%2 ⇒ 1 ≤ 2·y + -1·x) ∧ 1351234 ≥ y ∧ 1 ≥ x ∧ -1 ≤ 2·y + -1·x ∧ (2702470 ≤ x ∨ 2·y ≥ x ∨ -1 ≥ y)

In [4]:
sol_pre = And( Implies(x0 % 2 == 1, x0 == (2 * y0) -1),
               x0 <= 2 * y0,
               x0 >= 2*y0 - 1)

In [5]:
sol_post = substitute(sol_pre, [(x0, x1), (y0, y1)])

In [6]:
print(init.sexpr())
print(inductive_step.sexpr())
print(sol_pre.sexpr())
print(sol_post.children())


(and (= x0 0) (= y0 0))
(let ((a!1 (= y1 (ite (= (mod x0 2) 0) (+ y0 1) y0))))
  (and (= x1 (+ x0 1)) a!1))
(and (=> (>= y0 5000) (>= x0 y0)) (<= x0 y0) (>= y0 5000))
[Implies(y1 >= 5000, x1 >= y1), x1 <= y1, y1 >= 5000]


In [7]:
s_init = Solver()
s_init.add(init)
s_init.add(z3.Not(sol_pre))
s_init.check()

sat

In [8]:
s_tr = Solver()
s_tr.add(sol_pre, inductive_step, z3.Not(sol_post))
s_tr.check()

sat

In [12]:
m_tr = s_tr.model()
m_tr

[y0 = 5001, x0 = 5001, x1 = 5002, y1 = 5001]

In [10]:
sol_post.children()

[Implies(y1 >= 5000, x1 >= y1), x1 <= y1, y1 >= 5000]

In [11]:
list(map(lambda w: m_tr.eval(w) , sol_post.children()))

NameError: name 'm_tr' is not defined

In [ ]:
s_fail = Solver()
s_fail.add(sol_pre, bad)
s_fail.check()


unsat

In [ ]:
print(bad)
print(s_fail)

And(x0 == 2702470, Not(y0 == 1351235))
[And(Implies(x0%2 == 1, x0 == 2*y0 - 1),
     x0 <= 2*y0,
     x0 >= 2*y0 - 1),
 And(x0 == 2702470, Not(y0 == 1351235))]


In [ ]:
#m_f = s_fail.model()
#m_f

In [ ]:
sol_post.children()

[Implies(x1%2 == 1, x1 == 2*y1 - 1), x1 <= 2*y1, x1 >= 2*y1 - 1]

In [ ]:
list(map(lambda w: m_f.eval(w) , sol_post.children()))

[Or(Not(x1%2 == 1), x1 == -1 + 2*y1), x1 <= 2*y1, x1 >= -1 + 2*y1]